# 02_narration Prompt API Playground

Scope: API-only prompt testing for `harness/prompts/02_narration`.

This notebook uses a **hardcoded in-notebook plan JSON** and does not invoke the pipeline.

## Cell 1: Setup

Purpose:
- Resolve repo imports from notebook context.
- Load `.env` automatically.
- Import prompt template helpers and API client.

In [1]:
import json
import os
import sys
from pathlib import Path

# Resolve repo root whether cwd is repo root, notebooks/, or another child folder.
cwd = Path.cwd().resolve()
candidates = [cwd, *cwd.parents]
repo_root = None
for p in candidates:
    if (p / 'harness').exists() and (p / 'notebooks').exists():
        repo_root = p
        break
if repo_root is None:
    raise RuntimeError('Could not locate repo root containing harness/ and notebooks/')
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Load environment from repo .env if possible.
env_path = repo_root / '.env'
try:
    from dotenv import load_dotenv
    if env_path.exists():
        load_dotenv(env_path)
except Exception:
    pass

from harness.prompts import load_prompt_template, read_file, PROMPTS_DIR, TEMPLATES_DIR
from harness.client import LLMClient, estimate_tokens

print(f'Repo root: {repo_root}')

Repo root: /Users/velocityworks/IdeaProjects/flaming-horse


## Cell 2: Hardcoded `plan.json`

Purpose:
- Keep the full plan payload in-notebook for rapid prompt iteration.
- Edit this object directly to test narration behavior.

In [2]:
plan_data = {
    "title": "How Matrix Multiplication Works",
    "description": "A step-by-step guide to understanding matrix multiplication, starting from matrix basics and building to full examples with clear visualizations.",
    "target_duration_seconds": 360,
    "scenes": [
        {
            "id": "scene_01_intro",
            "title": "Introduction to Matrix Multiplication",
            "narration_key": "scene_01_intro",
            "description": "Introduce the topic, explain why matrix multiplication is important in math, computing, and graphics, and preview the video structure.",
            "estimated_duration_seconds": 25,
            "visual_ideas": [
                "Title text fades in with matrix grid icons",
                "Two sample matrices A (2x2) and B (2x2) slide in from sides",
                "Quick animation of C = A * B result matrix appearing"
            ]
        },
        {
            "id": "scene_02_matrix_basics",
            "title": "What is a Matrix?",
            "narration_key": "scene_02_matrix_basics",
            "description": "Define a matrix as a rectangular array of numbers, show notation like A = [rows x columns], and display simple examples.",
            "estimated_duration_seconds": 30,
            "visual_ideas": [
                "Empty grid fills with numbers to form 2x3 matrix A",
                "Label rows and columns with indices i,j",
                "Side-by-side: row vector, column vector, and full matrix"
            ]
        }
    ]
}

temperature = 0.7
max_tokens = 16000

## Cell 3: Compose Prompts

Purpose:
- Compose narration prompts using the same template files as harness.
- Uses the hardcoded plan object above (no project file reads).

In [3]:
system_prompt = load_prompt_template(
    'narration',
    'system.md',
    {
        'core_rules': read_file(PROMPTS_DIR / '_shared' / 'core_rules.md'),
        'pipeline_doc': read_file(TEMPLATES_DIR / 'manim_content_pipeline.md'),
    },
)

user_prompt = load_prompt_template(
    'narration',
    'user.md',
    {
        'title': plan_data.get('title', 'Unknown'),
        'plan_json': json.dumps(plan_data, indent=2),
    },
)

print(f'System prompt chars: {len(system_prompt):,}')
print(f'User prompt chars:   {len(user_prompt):,}')
print(f'System est tokens:   {estimate_tokens(system_prompt):,}')
print(f'User est tokens:     {estimate_tokens(user_prompt):,}')

System prompt chars: 494
User prompt chars:   2,314
System est tokens:   123
User est tokens:     578


## Cell 4: Prompt Preview

Purpose:
- Inspect rendered prompts before API call.

In [4]:
print('=== SYSTEM PROMPT PREVIEW ===')
print(system_prompt[:2000])
print('\n=== USER PROMPT PREVIEW ===')
print(user_prompt[:2000])

=== SYSTEM PROMPT PREVIEW ===
You are an expert educational voiceover writer.

Your job is to write scene-by-scene narration that is clear, accurate, and easy to follow.

Behavior:
- Prioritize conceptual clarity for beginners.
- Keep tone conversational but concise.
- Maintain continuity across scenes so the script feels like one coherent lesson.
- Match each scene's planned scope and approximate duration.
- Prefer concrete phrasing over abstract filler.
- Avoid stage directions or production notes in narration text.


=== USER PROMPT PREVIEW ===
Write the full narration for this video.

Title: How Matrix Multiplication Works

Plan JSON:
```json
{
  "title": "How Matrix Multiplication Works",
  "description": "A step-by-step guide to understanding matrix multiplication, starting from matrix basics and building to full examples with clear visualizations.",
  "target_duration_seconds": 360,
  "scenes": [
    {
      "id": "scene_01_intro",
      "title": "Introduction to Matrix Multipli

## Cell 5: API Call

Purpose:
- Send composed narration prompts to the LLM API.
- Print raw response text only.

In [5]:
provider = os.getenv('LLM_PROVIDER', 'XAI').upper()
api_key_var = f'{provider}_API_KEY'
assert os.getenv(api_key_var), f'Missing {api_key_var} in environment/.env'

client = LLMClient()
response_text = client.chat_completion(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    temperature=temperature,
    max_tokens=max_tokens,
)

print(response_text[:6000])

🤖 Harness using:
   Provider: XAI
   Base URL: https://api.x.ai/v1
   Model: grok-4-1-fast-reasoning
{"scene_01_intro":"Welcome to 'How Matrix Multiplication Works.' This key operation in math drives computer graphics, AI, and data analysis by transforming and combining data efficiently. We'll start with matrix basics, cover the strict rules for multiplying them, then work through full examples with visuals. Imagine sliding two grids together to form a new one, like this: A times B equals C. Let's dive in.","scene_02_matrix_basics":"First, what is a matrix? It's a rectangular array of numbers organized in rows and columns. We write it as A with dimensions m by n—m rows, n columns. Each number has position i for row, j for column, called element a_{i,j}. See this 2 by 3 matrix filling up. Compare it to a row vector—one row, multiple columns—a column vector—multiple rows, one column—and a full matrix like A here."}


## Cell 6: Response Validation

Purpose:
- Validate response is JSON object and check expected narration keys from the hardcoded plan.

In [6]:
expected_keys = [s.get('narration_key') for s in plan_data.get('scenes', []) if s.get('narration_key')]

parsed = json.loads(response_text)
assert isinstance(parsed, dict), 'Narration response must be a JSON object'

missing = [k for k in expected_keys if k not in parsed]
extra = [k for k in parsed.keys() if k not in expected_keys]
non_string_values = [k for k, v in parsed.items() if not isinstance(v, str)]

print(f'Expected keys: {len(expected_keys)}')
print(f'Response keys: {len(parsed)}')
print(f'Missing keys: {missing}')
print(f'Extra keys: {extra}')
print(f'Non-string values: {non_string_values}')

Expected keys: 2
Response keys: 2
Missing keys: []
Extra keys: []
Non-string values: []
